In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(1)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
import matplotlib.pyplot as plt
import PIL.Image as Image
from PIL import ImageOps
import math
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
import json
from tensorflow.keras.losses import mse
import os
import glob
import random
#import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
import datetime
from tensorflow.keras.layers import Dense, Input, concatenate, Dropout, Conv2D, MaxPooling2D, Flatten, Lambda, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import optimizers
%matplotlib inline

In [2]:
def euclidean_distance(y_true, y_pred):
    return K.sqrt(K.maximum(K.sum(K.square(y_pred - y_true), axis=-1, keepdims=True), K.epsilon()))

In [ ]:
# feature extraction from input image
img = Input(shape = (150,150,3), name="input_image")

x = Conv2D(64, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv1")(img)
x = Conv2D(64, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv2")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1')(x)
    
x = Conv2D(64, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv3")(x)
x = Conv2D(64, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv4")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2')(x)
   
x = Conv2D(128, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv5")(x)
x = Conv2D(128, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv6")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool3')(x)
    
x = Conv2D(128, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv7")(x)
x = Conv2D(128, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv8")(x)
x = BatchNormalization()(x)
    
x = Flatten()(x)
x = Dropout(0.75, noise_shape=None, seed=None)(x)
x = Dense(1024, name='FC1')(x)
out = Dense(13, name='loss')(x)

# create model
model = Model(inputs=[img], outputs=[out])

# set output types
target = tf.compat.v1.placeholder(dtype='float32', shape=(13,1)) 

# get model summary
#model.summary()

# compile model
model.compile(optimizer=optimizers.legacy.Adam(lr=0.0015, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss=euclidean_distance)
#plot_model(model, to_file='model.png')


In [8]:
import numpy as np

data_path = "your pth/"
Fx = np.load(data_path+"fx.npy", mmap_mode='r')
Fy = np.load(data_path+"fy.npy", mmap_mode='r')
U0 = np.load(data_path+"px.npy", mmap_mode='r')
V0 = np.load(data_path+"py.npy", mmap_mode='r')
Baseline = np.load(data_path+"baseline.npy", mmap_mode='r')
Disparity = np.load(data_path+"disparity.npy", mmap_mode='r')
Tx = np.load(data_path+"tx.npy", mmap_mode='r')
Ty = np.load(data_path+"ty.npy", mmap_mode='r')
Tz = np.load(data_path+"tz.npy", mmap_mode='r')
Pitch = np.load(data_path+"theta.npy", mmap_mode='r')
Xw = np.load(data_path+"X.npy", mmap_mode='r')
Yw = np.load(data_path+"Y.npy", mmap_mode='r')
Zw = np.load(data_path+"Z.npy", mmap_mode='r')
Left_images = np.load(data_path + 'Left_images.npy', mmap_mode='r')
Right_images = np.load(data_path + 'Right_images.npy', mmap_mode='r')

In [9]:
X = []
Y = []

for idx in range(len(Left_images)):
                
    l_im = Left_images[idx]
    #l_im = l_im[:,:,0]
    #r_im = Right_images[idx]
    #r_im = r_im[:,:,0]
    #training_image = np.dstack((l_im, r_im))
        
    X.append(l_im)
                
    Y.append([Fx[idx], Fy[idx], U0[idx], V0[idx], Baseline[idx], Disparity[idx], Tx[idx], Ty[idx], Tz[idx], Pitch[idx], Xw[idx], Yw[idx], Zw[idx]])
  

In [10]:
print ("dataset: ",np.shape(X))

dataset:  (5389, 150, 150, 3)


In [ ]:
import os
from keras.callbacks import TensorBoard, LearningRateScheduler
from keras.applications.inception_v3 import InceptionV3
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten, Input
#from utils_regressor_focal_dist import RotNetDataGenerator, angle_error, CustomModelCheckpoint
from keras import optimizers
import numpy as np
import glob, math
from shutil import copyfile
import datetime, random
import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session


model.load_weights('DeepFocal.h5')



input_shape = (112, 112, 2)


output = model.predict(
    x=[X],
    batch_size=32,
    verbose=1
)

In [12]:
error = {}

error["fx"] = 0
error["fy"] = 0
error["u0"] = 0
error["v0"] = 0
error["baseline"] = 0
error["disparity"] = 0
error["x"] = 0
error["y"] = 0
error["z"] = 0
error["pitch"] = 0
error["xworld"] = 0
error["yworld"] = 0
error["zworld"] = 0



key_counter = 0

for i in error.keys():
    
    k = 0

    for j  in range(np.shape(output)[0]):
        
        if key_counter == 0: 
    
            predicted_fx = output[j][key_counter]
            actual_fx = Y[k][key_counter]
    
            error[i] += abs(predicted_fx - actual_fx)
        
        if key_counter == 1: 
    
            predicted_fy = output[j][key_counter]
            actual_fy = Y[k][key_counter]
    
            error[i] += abs(predicted_fy - actual_fy)
        
        if key_counter == 2: 
    
            predicted_u0 = output[j][key_counter]
            actual_u0 = Y[k][key_counter]
    
            error[i] += abs(predicted_u0 - actual_u0)
        
        if key_counter == 3: 
    
            predicted_v0 = output[j][key_counter]
            actual_v0 = Y[k][key_counter]
    
            error[i] += abs(predicted_v0 - actual_v0)
        
        if key_counter == 4: 
    
            predicted_baseline = output[j][key_counter]
            actual_baseline = Y[k][key_counter]
    
            error[i] += abs(predicted_baseline - actual_baseline)
        
        if key_counter == 5: 
    
            predicted_disparity = output[j][key_counter]
            actual_disparity = Y[k][key_counter]
    
            error[i] += abs(predicted_disparity - actual_disparity)
        
        if key_counter == 6: 
    
            predicted_tx = output[j][key_counter]
            actual_tx = Y[k][key_counter]
    
            error[i] += abs(predicted_tx - actual_tx)
        
        if key_counter == 7: 
    
            predicted_ty = output[j][key_counter]
            actual_ty = Y[k][key_counter]
    
            error[i] += abs(predicted_ty - actual_ty)
        
        if key_counter == 8: 
    
            predicted_tz = output[j][key_counter]
            actual_tz = Y[k][key_counter]
    
            error[i] += abs(predicted_tz - actual_tz)
        
        if key_counter == 9: 
    
            predicted_pitch = output[j][key_counter]
            actual_pitch = Y[k][key_counter]
    
            error[i] += abs(predicted_pitch - actual_pitch)
        
        if key_counter == 10: 
    
            predicted_x = output[j][key_counter]
            actual_x = Y[k][key_counter]
    
            error[i] += abs(predicted_x - actual_x)
        
        if key_counter == 11: 
    
            predicted_y = output[j][key_counter]
            actual_y = Y[k][key_counter]
    
            error[i] += abs(predicted_y - actual_y)
        
        if key_counter == 12: 
    
            predicted_z = output[j][key_counter]
            actual_z = Y[k][key_counter]
    
            error[i] += abs(predicted_z - actual_z)
        
        k += 1
    
    
    error[i] /= np.shape(output)[0]
    
    key_counter += 1

print (error)

{'fx': 57.78863762455408, 'fy': 48.36515494944943, 'u0': 8.47541927649816, 'v0': 8.949758482589905, 'baseline': 0.6086184943851279, 'disparity': 45.03145151584159, 'x': 1.9912913411247029, 'y': 1.552423298598099, 'z': 0.5420030601898371, 'pitch': 0.7369801352287763, 'xworld': 13207.065497511036, 'yworld': 532938.1388990598, 'zworld': 399379.2591479133}
